In [1]:
import pandas as pd
import os

1. Pré-processamento

In [2]:
arquivos = {
    os.path.join("..", "Área de Culturas Permanentes (hectares)", "Área colhida permanente 1999-2020.csv"): "df_permanente",
    os.path.join("..", "Producao-Milhos", "Produção - milho(R$).xls"): "df_milho",
    os.path.join("..", "Producao-Soja", "Produção - soja (R$).csv"): "df_soja",
    os.path.join("..", "Despesa Orçamentária (R$)", "Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls"): "df_despesa",
    os.path.join("..", "Focos de Incendio", "firefoci.csv"): "df_incendio",
    os.path.join("..", "Área de Culturas Temporárias (hectares)", "Área colhida temporária 1999-2020.csv"): "df_temporaria",
    os.path.join("..", "Área Plantada ou Colhida (hectares)", "Área plantada ou destinada à colheita - total.xls"): "df_plantada_colhida",
    os.path.join("..", "PIB", "PIB 1999-2020.csv"): "df_pib",
    os.path.join("..", "Valor da Produção Agrícola (R$)", "Valor de Produção 1999-2020.csv"): "df_producao" 
}

# Lista dos estados que queremos manter
estados = ["Acre", "Amazonas", "Amapá", "Maranhão", "Pará", "Roraima", "Tocantins", "Rondônia", "Mato Grosso"] 

# Lista de anos como strings
anos_str = [str(year) for year in range(1999, 2021)]

# Configurar Pandas para exibir números inteiros completos
pd.options.display.float_format = '{:.00f}'.format  

# Criar variáveis para armazenar os DataFrames
for arquivo, nome_df in arquivos.items():
    try:
        # Ler CSV com separador ','
        try:
            df = pd.read_csv(arquivo, sep=",")
        except:
            df = pd.read_excel(arquivo)

        # Normalizar os nomes das colunas (remover espaços extras)
        df.columns = df.columns.str.strip()

        # Remover colunas "Código" e "Sigla", se existirem
        colunas_para_remover = [col for col in ["Codigo", "Código", "Sigla"] if col in df.columns]
        if colunas_para_remover:
            df = df.drop(columns=colunas_para_remover)
            print(f"🗑️ Colunas removidas de {nome_df}: {colunas_para_remover}")

        # Filtrar os estados desejados
        if "Estado" in df.columns:
            df = df[df["Estado"].isin(estados)]
            print(f"🌎 Filtrados apenas os estados desejados em {nome_df}")

        # Armazenar na variável correta
        globals()[nome_df] = df
        print(f"✔ {arquivo} carregado como {nome_df}")
        
    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Exibir quais DataFrames foram carregados
print("\n📂 DataFrames disponíveis:", list(arquivos.values()))

🗑️ Colunas removidas de df_permanente: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_permanente
✔ ..\Área de Culturas Permanentes (hectares)\Área colhida permanente 1999-2020.csv carregado como df_permanente
🗑️ Colunas removidas de df_milho: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_milho
✔ ..\Producao-Milhos\Produção - milho(R$).xls carregado como df_milho
🗑️ Colunas removidas de df_soja: ['Sigla']
🌎 Filtrados apenas os estados desejados em df_soja
✔ ..\Producao-Soja\Produção - soja (R$).csv carregado como df_soja
🗑️ Colunas removidas de df_despesa: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_despesa
✔ ..\Despesa Orçamentária (R$)\Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls carregado como df_despesa
✔ ..\Focos de Incendio\firefoci.csv carregado como df_incendio
🗑️ Colunas removidas de df_temporaria: ['Código', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_

2. Transformação formato Wide para Long

Com exceção do de Focos de Incêndio, os DataFrames estão em formato longo, com 22 colunas representando os anos de 1999 a 2020. A função .melt() permite transformar essas colunas em linhas, facilitando a análise e manipulação dos dados.

In [3]:
df_permanente = df_permanente.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="Culturas P. Colhidas (hec)")
df_permanente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Estado                      198 non-null    object
 1   Ano                         198 non-null    object
 2   Culturas P. Colhidas (hec)  198 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [4]:
df_milho = df_milho.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="Producao de Milho (R$)")
df_milho.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Estado                  198 non-null    object 
 1   Ano                     198 non-null    object 
 2   Producao de Milho (R$)  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


In [5]:
df_soja = df_soja.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="Producao de Soja (R$)")
df_soja

,Estado,Ano,Producao de Soja (R$)
0,Acre,1999,136
1,Amazonas,1999,875
2,Amapá,1999,NaN
3,Maranhão,1999,243966
4,Mato Grosso,1999,3470093
...,...,...,...
193,Mato Grosso,2020,22939332
194,Pará,2020,1428549
195,Rondônia,2020,884772
196,Roraima,2020,162263


In [6]:
df_despesa = df_despesa.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="Despesa Orçamentária (R$)")
df_despesa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Estado                     198 non-null    object 
 1   Ano                        198 non-null    object 
 2   Despesa Orçamentária (R$)  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


In [7]:
df_incendio.rename(columns={"Total": "Acre"}, inplace=True)
df_incendio.rename(columns={"Anos": "Ano"}, inplace=True)
df_incendio["Ano"] = df_incendio["Ano"].astype(str)
df_incendio = df_incendio.melt(id_vars=["Ano"], value_vars=estados, var_name="Estado", value_name="Focos de Incêndio")
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ano                198 non-null    object 
 1   Estado             198 non-null    object 
 2   Focos de Incêndio  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


In [8]:
df_temporaria = df_temporaria.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="Culturas T. Colhidas (hec)")
df_temporaria.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Estado                      198 non-null    object
 1   Ano                         198 non-null    object
 2   Culturas T. Colhidas (hec)  198 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [9]:
df_plantada_colhida = df_plantada_colhida.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="Área Plantada ou Colhida (hec)")
df_plantada_colhida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Estado                          198 non-null    object
 1   Ano                             198 non-null    object
 2   Área Plantada ou Colhida (hec)  198 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [10]:
df_pib = df_pib.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="PIB Estadual")
df_pib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Estado        198 non-null    object 
 1   Ano           198 non-null    object 
 2   PIB Estadual  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


In [11]:
df_producao = df_producao.melt(id_vars=["Estado"], value_vars=anos_str, var_name="Ano", value_name="PIB Agrícula Estadual")
df_producao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Estado                 198 non-null    object 
 1   Ano                    198 non-null    object 
 2   PIB Agrícula Estadual  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


2. Merge em um unico DataFrame

In [12]:
df_final = df_permanente.merge(df_milho, how='outer', on=["Ano", "Estado"])\
                            .merge(df_soja, how='outer', on=["Ano", "Estado"] )\
                            .merge(df_despesa, how='outer', on=["Ano", "Estado"])\
                            .merge(df_temporaria, how='outer', on=["Ano", "Estado"])\
                            .merge(df_plantada_colhida, how='outer', on=["Ano", "Estado"])\
                            .merge(df_pib, how='outer', on=["Ano", "Estado"])\
                            .merge(df_producao, how='outer', on=["Ano", "Estado"])\
                            .merge(df_incendio, how='outer', on=["Ano", "Estado"])
df_final

,Estado,Ano,Culturas P. Colhidas (hec),Producao de Milho (R$),Producao de Soja (R$),Despesa Orçamentária (R$),Culturas T. Colhidas (hec),Área Plantada ou Colhida (hec),PIB Estadual,PIB Agrícula Estadual,Focos de Incêndio
0,Acre,1999,9864,18905,136,16480351,81922,91787,1866542,280901,347
1,Amapá,1999,1283,664,NaN,2869222,6903,8235,2128307,54019,101
2,Amazonas,1999,61397,12572,875,17685970,150055,213964,13933920,623336,1048
3,Maranhão,1999,28896,96561,243966,15239851,1130797,1180275,10308256,1174349,8712
4,Mato Grosso,1999,91480,287455,3470093,43232165,4259560,4445242,12364905,6462378,44307
...,...,...,...,...,...,...,...,...,...,...,...
193,Mato Grosso,2020,37589,10090286,22939332,555308252,17149919,17188641,178649564,41799931,47708
194,Pará,2020,651600,346957,1428549,380705994,1310866,1968093,215935604,7981859,38603
195,Rondônia,2020,83545,396740,884772,210685708,721826,813919,51598741,2196163,11145
196,Roraima,2020,10007,47138,162263,67296397,84045,94052,16024276,422453,1930


In [13]:
df_final.to_csv("tabela_final", index = False, encoding="utf-8")
df_path = os.getcwd()

print("📂 Dataframe salvo em: ", df_path) 

📂 Dataframe salvo em:  d:\studs\git\LabSesing-case-study\Código
